In [24]:
from langchain.chains.question_answering.map_reduce_prompt import messages
!pip install python-dotenv
!pip install -U openai langchain langchain-openai
!pip install langchain_community
!pip install pypdf
!pip install faiss-cpu

ModuleNotFoundError: No module named 'pydantic'

In [14]:
from dotenv import load_dotenv
import os
import openai
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_openai import ChatOpenAI
# from langchain.embeddings import HuggingFaceEmbeddings
from langchain_openai import OpenAIEmbeddings
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import FAISS

In [15]:
def env_loader():
    load_dotenv()
    openai.api_key = os.getenv("OPENAI_API_KEY")

In [16]:
def pdf_loader(file_path):
    loader = PyPDFLoader(file_path)
    return loader

In [17]:
def show_pdf(loader):
    documents = loader.load()

    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    docs = text_splitter.split_documents(documents)
    context = [doc.page_content for doc in docs]

    print(context)

In [18]:
def create_vectorstore(loader):
    embeddings = OpenAIEmbeddings()
    #embeddings = HuggingFaceEmbeddings()

    index = VectorstoreIndexCreator(
        vectorstore_cls=FAISS,
        embedding=embeddings,
        ).from_loaders([loader])

    # 파일로 저장
    index.vectorstore.save_local("vectorstore")

    return index

In [38]:
def set_model(model):

    chat = ChatOpenAI(
        model=model,
        temperature=0.3,
        max_tokens=512,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
    )

    return chat

In [36]:
def chat_setup():
    file_path = "C:/Users/user/Documents/GitLab/mammoth_chat_Langchain/doc/타임소프트 회사소개서.pdf"

    env_loader()
    loader = pdf_loader(file_path)
    # show_pdf(loader)
    index = create_vectorstore(loader)
    chat = set_model(model='gpt-4o-mini')

    return chat, index

In [47]:
def chat_input(index, user_input, chat):

    system_msg = """
        당신은 친절한 기업 분석가입니다.
        주어진 데이터를 보고 사용자에게 정확하게 대답하세요.
        사용자가 데이터에 없는 내용을 질문 하면, 해당 문제에 대해 정보가 부족합니다. 라고 대답하세요.
        """

    messages = [
        {
            "rol": "system",
            "content": system_msg,
        },
        {
            "rol": "user",
            "content": user_input,
        },
    ]

    return index.query( messages, llm=chat, )

In [49]:
while True:
    chat, index = chat_setup()

    user_input = input("you: ")
    print(f"you: {user_input}")

    if user_input == "exit":
        break
    elif user_input.replace(" ", "") == "":
        print("chat: 질문이 없습니다. 정확한 질문을 해주시면 답변해 드리겠습니다.")
    else:
        chat_text = chat_input(
            chat=chat,
            index=index,
            user_input=user_input,
        )
        print(f"chat: {chat_text}")

you: 타임 소프트는 우주관련 산업도 해?


TypeError: argument 'text': 'list' object cannot be converted to 'PyString'

In [ ]:
타임 소프트는 우주관련 산업도 해?